In [1]:
import json
import pandas as pd

In [2]:
# Load data from the JSON file
with open("final_data/patents_data_v2_3_final1.json", "r") as json_file:
    data = json.load(json_file)

In [3]:
# Load data from the JSON file
with open("final_data/patents_data_v3_no_citations.json", "r") as json_file:
    data_2 = json.load(json_file)

In [4]:
# Count entries in `data`
num_entries = len(data)
num_entries_2 = len(data_2)

print(f"Number of entries in data: {num_entries}")
print(f"Number of entries in data_2: {num_entries_2}")

Number of entries in data: 53411
Number of entries in data_2: 7989


In [5]:
data.extend(data_2)

In [93]:
entry = data[14756]

In [95]:
# Check if 'description' field is present and contains 'text'
description_data = entry.get('description', {}).get('text')
# Check if the language is 'en'
#desc_lang= entry['description']['lang']

In [96]:
# claims_data = entry.get('claims')
# claims_lang = claims_data[0].get('lang')
# claim_text = ' '.join([claim['claim_text'][0] for claim in claims_data[0]['claims']])

In [97]:
claims_data = entry.get('claims')
claims_lang = claims_data[0].get('lang')

claim_texts = []
for claim in claims_data[0]['claims']:
    # Handle both cases: list of strings or a single string
    if isinstance(claim['claim_text'], list):
        claim_texts.append(' '.join(claim['claim_text']))
    else:
        claim_texts.append(claim['claim_text'])

claim_text = ' '.join(claim_texts)

In [125]:
invention_titles = entry['biblio']['invention_title']
for title in invention_titles:
    if title['lang'] == 'en':  # Check if the language is English
        title_data = title['text']

In [126]:
entry['biblio']['publication_reference']
pub_number = entry['biblio']['publication_reference']['jurisdiction'] + entry['biblio']['publication_reference']['doc_number']+ entry['biblio']['publication_reference']['kind']

In [ ]:
lens_id = entry.get('lens_id') 

In [87]:
df_temp_all = pd.DataFrame(columns=['lens_id','publication_number', 'title', 'claims', 'description'])
counter = 0  # Initialize a counter

for entry in data:
    # Check if 'description' field is present and contains 'text'
    description_data = entry.get('description', {}).get('text')
    # Check if the language is 'en'
    # desc_lang= entry['description']['lang']

    lens_id = entry.get('lens_id') 

    claims_data = entry.get('claims')
    claims_lang = claims_data[0].get('lang')

    claim_texts = []
    for claim in claims_data[0]['claims']:
        # Handle both cases: list of strings or a single string
        if isinstance(claim['claim_text'], list):
            claim_texts.append(' '.join(claim['claim_text']))
        else:
            claim_texts.append(claim['claim_text'])

    claim_text = ' '.join(claim_texts)

    invention_titles = entry['biblio']['invention_title']
    for title in invention_titles:
        if title['lang'] == 'en':  # Check if the language is English
            title_data = title['text']
    
    pub_number = entry['biblio']['publication_reference']['jurisdiction'] + entry['biblio']['publication_reference']['doc_number']+ entry['biblio']['publication_reference']['kind']

    df_temp = pd.DataFrame([{
        'lens_id': lens_id,
        'publication_number': pub_number,
        'title': title_data,
        'claims': claim_text,
        # 'claims_lang': claims_lang,
        'description': description_data
        # 'description_lang': desc_lang
    }])

    df_temp_all = pd.concat([df_temp_all, df_temp], ignore_index=True)
    counter += 1
    if counter % 10000 == 0:
        print(counter)



10000
20000
30000
40000
50000
60000


In [ ]:
#to_excel("final_database.xlsx", index=False, engine='openpyxl')

#### Check Claims

In [89]:
# Count cells with fewer than 30 characters
count_short_text = (df_temp_all['claims'].str.len() < 40).sum()

print(f"Number of cells with fewer than 30 characters: {count_short_text}")

Number of cells with fewer than 30 characters: 4


In [92]:
# Identify cells with fewer than 30 characters
short_text_mask = df_temp_all['claims'].str.len() < 50

# Display rows with short text
short_text_rows = df_temp_all[short_text_mask]

print("Rows with less than 30 characters:")
print(short_text_rows)

Rows with less than 30 characters:
                   lens_id publication_number  \
14756  114-125-945-337-503     WO2023173236A1   
19204  180-726-891-119-451     WO2001066941A1   
37658  071-882-095-067-905     WO2024159337A1   
46872  014-978-483-745-949    US20060061105A1   

                                                   title  \
14756  WIND TURBINE INTEGRATED ASSEMBLY AND OPERATING...   
19204  NON-UTILITY POWER GENERATOR BY WIND POWER GENE...   
37658  METHOD FOR UTILIZING INFLATABLE STRUCTURE IN W...   
46872                           Wind electricity vehicle   

                                        claims  \
14756           风力发电机的轴线平行安装和运行。 风力发电机成串安装和运行。   
19204  5冃 求 の 車 11 高層ビル （30米以上） に設置する風力による発電装置   
37658    充气升空气球结构上安装敷设太阳能光伏电池 充气结构做成垂直轴风力发电机叶轮   
46872         1 . I claim an electric vehicle.   

                                             description  
14756  [根据细则26改正 08.07.2022] 技术领域：低碳减排，风力发电。 [根据细则26改...  
19204  明 細 書 中高層ビルの屋上に設置する風力発電による自家発電装置 技術分野 この発明

#### Translation

In [2]:
from langdetect import detect, DetectorFactory, LangDetectException
import time
from IPython.display import display, clear_output
from googletrans import Translator

In [3]:
def translate_if_non_english(text, is_non_english, src_lang='auto', dest_lang='en'):
    if is_non_english:
        try:
            # Translate with the specified source language (e.g., 'es' for Spanish, 'fr' for French)
            translated = translator.translate(text, src=src_lang, dest=dest_lang)
            return translated.text
        except Exception as e:
            print(f"Error translating text: {e}")
            return text  # Return the original text if translation fails
    return text  # Return the original text if already English

In [4]:
def is_non_english(text):
    try:
        # Detect language and check if it's not English ('en')
        return detect(text) != 'en'
    except LangDetectException:
        # Handle cases with empty or unrecognizable text
        return True

In [5]:
translator = Translator()

#### Claims Translation 1

In [ ]:
# 7 min run time
# Function to detect non-English cells

# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish'] = df_temp_all['claims'].apply(is_non_english)

# Count non-English cells
non_english_count = df_temp_all['IsNonEnglish'].sum()

print(f"Number of non-English cells: {non_english_count}")

Number of non-English cells: 10406


In [ ]:
# 164 min run time

# Apply translation only to non-English rows
df_temp_all['claims_translated'] = df_temp_all.apply(
    lambda row: translate_if_non_english(row['claims'], row['IsNonEnglish']),
    axis=1
)

In [105]:
df_temp_all.to_excel('output_file_1_only_claims_trans.xlsx', index=False)

#### Second translation

In [ ]:
# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish2'] = df_temp_all['claims_translated'].apply(is_non_english)

# Count non-English cells
non_english_count_claims_2 = df_temp_all['IsNonEnglish2'].sum()

print(f"Number of non-English cells: {non_english_count}")

Number of non-English cells: 4104


In [117]:
print(f"Total non-English rows to translate: {non_english_count_claims_2}")

# Start timer
start_time = time.time()

# Apply translation with a countdown
remaining_rows = non_english_count_claims_2  # Initialize countdown
def translate_and_countdown(row):
    global remaining_rows
    if row['IsNonEnglish2']:
        result = translate_if_non_english(row['claims_translated'], row['IsNonEnglish2'])
        remaining_rows -= 1  # Decrease count
        
        # Clear previous output and display the updated remaining rows
        clear_output(wait=True)
        display(f"Remaining rows to process: {remaining_rows}")
        return result
    return row['claims_translated']

df_temp_all['claims_translated_2'] = df_temp_all.apply(translate_and_countdown, axis=1)

# End timer
end_time = time.time()
print(f"Translation completed in {end_time - start_time:.2f} seconds.")

'Remaining rows to process: 0'

Translation completed in 2349.34 seconds.


In [ ]:
# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish3'] = df_temp_all['claims_translated_2'].apply(is_non_english)

# Count non-English cells
non_english_count_claims_2 = df_temp_all['IsNonEnglish3'].sum()

print(f"Number of non-English cells: {non_english_count_claims_2}")

Number of non-English cells: 4104


In [122]:
print(f"Total non-English rows to translate: {non_english_count_claims_2}")

# Start timer
start_time = time.time()

# Apply translation with a countdown
remaining_rows = non_english_count_claims_2  # Initialize countdown
def translate_and_countdown(row):
    global remaining_rows
    if row['IsNonEnglish3']:
        result = translate_if_non_english(row['claims_translated_2'], row['IsNonEnglish3'])
        remaining_rows -= 1  # Decrease count
        
        # Clear previous output and display the updated remaining rows
        clear_output(wait=True)
        display(f"Remaining rows to process: {remaining_rows}")
        return result
    return row['claims_translated_2']

df_temp_all['claims_translated_3'] = df_temp_all.apply(translate_and_countdown, axis=1)

# End timer
end_time = time.time()
print(f"Translation completed in {end_time - start_time:.2f} seconds.")

'Remaining rows to process: 0'

Translation completed in 749.57 seconds.


In [123]:
# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish3'] = df_temp_all['claims_translated_3'].apply(is_non_english)

# Count non-English cells
non_english_count_claims_3 = df_temp_all['IsNonEnglish3'].sum()

print(f"Number of non-English cells: {non_english_count_claims_3}")

Number of non-English cells: 2901


In [ ]:
df_temp_all = df_temp_all.drop(columns=['IsNonEnglish', 'claims_translated', 'IsNonEnglish2', 'claims', 'claims_translated_2'], inplace=False)


In [136]:
df_temp_all = df_temp_all.rename(columns={'claims_translated_3': 'claims'})

In [133]:
df_temp_all = df_temp_all.rename(columns={'IsNonEnglish3': 'IsNonEnglish_claims'})

In [137]:
df_temp_all

,lens_id,publication_number,title,description,IsNonEnglish_desc,IsNonEnglish_claims,claims
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,BLADE MOUNTING The present invention concerns ...,False,False,CLAIMS: 1. An arrangement for in situ mounting...
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,TECHNICAL FIELD The present invention relates ...,False,False,1 . A speed reducer used in a yaw drive appara...
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,False,False,A wind turbine (1) having a rotor (4) with at ...
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,False,False,Method of updating software and/or firmware of...
4,000-664-023-248-319,EP2434144B1,Rotor blade or rotor blade segment for a wind ...,Die Erfindung betrifft ein Rotorblatt oder ein...,True,True,System mit einem Rotorblatt (10) oder einem Ro...
...,...,...,...,...,...,...,...
61395,196-473-373-722-750,EP2708743A1,WINDMILL,Technical Field The present invention relates ...,False,False,A wind turbine comprising:\n a rotor head; \n ...
61396,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,FIELD OF THE INVENTION The present invention r...,False,False,A method for installing a tower (1) formed by ...
61397,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,BACKGROUND Embodiments of the present specific...,False,False,"1 . An armature for an electrical machine, com..."
61398,198-470-700-069-007,EP3376025A1,WIND TURBINE VARIABLE-PITCH CONTROL METHOD AND...,The present application claims priority to Chi...,False,False,"A pitch control method for a wind turbine, com..."


#### Description Translation

In [ ]:
#15min run time
# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish_desc'] = df_temp_all['description'].apply(is_non_english)

# Count non-English cells
non_english_count_desc = df_temp_all['IsNonEnglish_desc'].sum()

print(f"Number of non-English cells: {non_english_count}")

Number of non-English cells: 10406


In [138]:

# # Function to translate non-English text
# def translate_if_non_english(text, is_non_english):
#     if is_non_english:
#         try:
#             translated = translator.translate(text, dest='en')  # Translate to English
#             return translated.text
#         except Exception as e:
#             return text  # Return original text if translation fails
#     return text  # Return original text if already English

def translate_if_non_english(text, is_non_english, src_lang='auto', dest_lang='en'):
    if is_non_english:
        try:
            # Translate with the specified source language (e.g., 'es' for Spanish, 'fr' for French)
            translated = translator.translate(text, src=src_lang, dest=dest_lang)
            return translated.text
        except Exception as e:
            print(f"Error translating text: {e}")
            return text  # Return the original text if translation fails
    return text  # Return the original text if already English


print(f"Total non-English rows to translate: {non_english_count_desc}")

# Start timer
start_time = time.time()

# Apply translation with a countdown
remaining_rows = non_english_count_desc  # Initialize countdown
def translate_and_countdown(row):
    global remaining_rows
    if row['IsNonEnglish_desc']:
        result = translate_if_non_english(row['description'], row['IsNonEnglish_desc'])
        remaining_rows -= 1  # Decrease count
        
        # Clear previous output and display the updated remaining rows
        clear_output(wait=True)
        display(f"Remaining rows to process: {remaining_rows}")
        return result
    return row['description']

df_temp_all['description_translated'] = df_temp_all.apply(translate_and_countdown, axis=1)

# End timer
end_time = time.time()
print(f"Translation completed in {end_time - start_time:.2f} seconds.")

'Remaining rows to process: 0'

Translation completed in 4265.06 seconds.


In [139]:
df_temp_all.to_excel('output_file_2_claimsX3_descX1.xlsx', index=False)

#### 2nd translation description

In [6]:
df_temp_all = pd.read_excel('output_file_2_claimsX3_descX1.xlsx')

In [7]:
df_temp_all 

,lens_id,publication_number,title,description,IsNonEnglish_desc,IsNonEnglish_claims,claims,description_translated
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,BLADE MOUNTING The present invention concerns ...,False,False,CLAIMS: 1. An arrangement for in situ mounting...,BLADE MOUNTING The present invention concerns ...
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,TECHNICAL FIELD The present invention relates ...,False,False,1 . A speed reducer used in a yaw drive appara...,TECHNICAL FIELD The present invention relates ...
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,False,False,A wind turbine (1) having a rotor (4) with at ...,The present invention relates to a wind turbin...
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,False,False,Method of updating software and/or firmware of...,Field of invention The present invention relat...
4,000-664-023-248-319,EP2434144B1,Rotor blade or rotor blade segment for a wind ...,Die Erfindung betrifft ein Rotorblatt oder ein...,True,True,System mit einem Rotorblatt (10) oder einem Ro...,Die Erfindung betrifft ein Rotorblatt oder ein...
...,...,...,...,...,...,...,...,...
61395,196-473-373-722-750,EP2708743A1,WINDMILL,Technical Field The present invention relates ...,False,False,A wind turbine comprising:\n a rotor head; \n ...,Technical Field The present invention relates ...
61396,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,FIELD OF THE INVENTION The present invention r...,False,False,A method for installing a tower (1) formed by ...,FIELD OF THE INVENTION The present invention r...
61397,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,BACKGROUND Embodiments of the present specific...,False,False,"1 . An armature for an electrical machine, com...",BACKGROUND Embodiments of the present specific...
61398,198-470-700-069-007,EP3376025A1,WIND TURBINE VARIABLE-PITCH CONTROL METHOD AND...,The present application claims priority to Chi...,False,False,"A pitch control method for a wind turbine, com...",The present application claims priority to Chi...


In [8]:
#15min run time
# Apply the function to detect non-English cells
df_temp_all['IsNonEnglish_desc_2'] = df_temp_all['description_translated'].apply(is_non_english)

# Count non-English cells
non_english_count_desc_2 = df_temp_all['IsNonEnglish_desc_2'].sum()

print(f"Number of non-English cells: {non_english_count_desc_2}")

Number of non-English cells: 9972


In [11]:
# List to store indices of failed translations
failed_indices = []

# Function to translate text and validate the output
def translate_if_non_english(text, is_non_english, row_index, src_lang='auto', dest_lang='en'):
    if is_non_english:
        try:
            # Translate with the specified source and destination language
            translated = translator.translate(text, src=src_lang, dest=dest_lang)
            
            # Validate translation: ensure the output is not identical to input
            if translated.text.strip().lower() == text.strip().lower():
                print(f"Translation failed for row index: {row_index}")
                failed_indices.append(row_index)  # Add index to failed list
                return text  # Optionally return the original text
            
            return translated.text
        except Exception as e:
            print(f"Error translating row index {row_index}: {e}")
            failed_indices.append(row_index)  # Add index to failed list
            return text  # Return the original text if translation fails
    return text  # Return the original text if already English

# Start translation process
print(f"Total non-English rows to translate: {non_english_count_desc_2}")

# Start timer
start_time = time.time()

# Initialize countdown
remaining_rows = non_english_count_desc_2

def translate_and_validate(row):
    global remaining_rows
    
    # Check if translation is needed
    if row['IsNonEnglish_desc_2']:
        # Perform translation and validate
        translated_text = translate_if_non_english(row['description_translated'], row['IsNonEnglish_desc_2'], row.name)
        remaining_rows -= 1  # Decrease count

        # Clear previous output and display progress
        clear_output(wait=True)
        display(f"Remaining rows to process: {remaining_rows}")
        return translated_text

    # Return the original description if translation is not needed
    return row['description_translated']

# Apply translation function
df_temp_all['description_translated_2'] = df_temp_all.apply(translate_and_validate, axis=1)

# End timer
end_time = time.time()

# Print the failed indices
print(f"Translation completed in {end_time - start_time:.2f} seconds.")
print(f"Number of failed translations: {len(failed_indices)}")

'Remaining rows to process: 0'

Translation completed in 3995.34 seconds.
Number of failed translations: 9890


In [10]:
df_temp_all

,lens_id,publication_number,title,description,IsNonEnglish_desc,IsNonEnglish_claims,claims,description_translated,IsNonEnglish_desc_2
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,BLADE MOUNTING The present invention concerns ...,False,False,CLAIMS: 1. An arrangement for in situ mounting...,BLADE MOUNTING The present invention concerns ...,False
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,TECHNICAL FIELD The present invention relates ...,False,False,1 . A speed reducer used in a yaw drive appara...,TECHNICAL FIELD The present invention relates ...,False
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,False,False,A wind turbine (1) having a rotor (4) with at ...,The present invention relates to a wind turbin...,False
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,False,False,Method of updating software and/or firmware of...,Field of invention The present invention relat...,False
4,000-664-023-248-319,EP2434144B1,Rotor blade or rotor blade segment for a wind ...,Die Erfindung betrifft ein Rotorblatt oder ein...,True,True,System mit einem Rotorblatt (10) oder einem Ro...,Die Erfindung betrifft ein Rotorblatt oder ein...,True
...,...,...,...,...,...,...,...,...,...
61395,196-473-373-722-750,EP2708743A1,WINDMILL,Technical Field The present invention relates ...,False,False,A wind turbine comprising:\n a rotor head; \n ...,Technical Field The present invention relates ...,False
61396,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,FIELD OF THE INVENTION The present invention r...,False,False,A method for installing a tower (1) formed by ...,FIELD OF THE INVENTION The present invention r...,False
61397,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,BACKGROUND Embodiments of the present specific...,False,False,"1 . An armature for an electrical machine, com...",BACKGROUND Embodiments of the present specific...,False
61398,198-470-700-069-007,EP3376025A1,WIND TURBINE VARIABLE-PITCH CONTROL METHOD AND...,The present application claims priority to Chi...,False,False,"A pitch control method for a wind turbine, com...",The present application claims priority to Chi...,False


In [14]:
import time

# List to store indices of failed translations
failed_indices = []

# Function to translate text and validate the output
def translate_if_non_english(text, is_non_english, row_index, src_lang='auto', dest_lang='en'):
    if is_non_english:
        try:
            # Add delay to limit to 5 requests per second
            time.sleep(0.2)  # 1 second / 5 requests = 0.2 seconds delay between requests
            
            # Translate with the specified source and destination language
            translated = translator.translate(text, src=src_lang, dest=dest_lang)
            
            # Validate translation: ensure the output is not identical to input
            if translated.text.strip().lower() == text.strip().lower():
                print(f"Translation failed for row index: {row_index}")
                failed_indices.append(row_index)  # Add index to failed list
                return text  # Optionally return the original text
            
            return translated.text
        except Exception as e:
            print(f"Error translating row index {row_index}: {e}")
            failed_indices.append(row_index)  # Add index to failed list
            return text  # Return the original text if translation fails
    return text  # Return the original text if already English

# Start translation process
print(f"Total non-English rows to translate: {non_english_count_desc_2}")

# Start timer
start_time = time.time()

# Initialize countdown
remaining_rows = non_english_count_desc_2

def translate_and_validate(row):
    global remaining_rows
    
    # Check if translation is needed
    if row['IsNonEnglish_desc_2']:
        # Perform translation and validate
        translated_text = translate_if_non_english(row['description_translated'], row['IsNonEnglish_desc_2'], row.name)
        remaining_rows -= 1  # Decrease count
        Len_failed_indices = len(failed_indices)
        # Clear previous output and display progress
        clear_output(wait=True)
        display(f"Remaining rows to process: {remaining_rows} and failed: {Len_failed_indices}")
        return translated_text

    # Return the original description if translation is not needed
    return row['description_translated']

# Apply translation function
df_temp_all['description_translated_2'] = df_temp_all.apply(translate_and_validate, axis=1)

# End timer
end_time = time.time()

# Print the failed indices
print(f"Translation completed in {end_time - start_time:.2f} seconds.")
print(f"Number of failed translations: {len(failed_indices)}")


'Remaining rows to process: 9918 and failed: 54'

KeyboardInterrupt: 

In [13]:
# Function to check if the text is English
def check_and_update_language(row):
    if row['IsNonEnglish_desc_2']:  # Only check for rows that are marked as non-English
        try:
            # Detect the language of the text
            detected_lang = detect(row['description_translated'])
            
            # If the detected language is English (en), change IsNonEnglish_desc_2 to False
            if detected_lang == 'en':
                return False  # Change to False as the text is in English
            else:
                return True  # Keep as True, since the text is not in English
        except Exception as e:
            print(f"Error detecting language for row index {row.name}: {e}")
            return row['IsNonEnglish_desc_2']  # If error, return the original value (True)
    return row['IsNonEnglish_desc_2']  # If it's already English, leave it as is

# Apply the check and update function to the DataFrame
df_temp_all['IsNonEnglish_desc_2'] = df_temp_all.apply(check_and_update_language, axis=1)

# Print updated dataframe or any relevant information
print(df_temp_all[['description_translated', 'IsNonEnglish_desc_2']].head())

                              description_translated  IsNonEnglish_desc_2
0  BLADE MOUNTING The present invention concerns ...                False
1  TECHNICAL FIELD The present invention relates ...                False
2  The present invention relates to a wind turbin...                False
3  Field of invention The present invention relat...                False
4  Die Erfindung betrifft ein Rotorblatt oder ein...                 True


In [29]:
from translate import Translator
from langdetect import detect
import re

# Initialize the translator (no API key needed)
translator = Translator(to_lang="en")  # Translate to English

# Access the 5th row (index 4) of 'description_translated' column and inspect the full row
row_4 = df_temp_all.loc[4]
print(f"Full content of row 4: {row_4}")

# Access the text to translate from the 'description_translated' column
text_to_translate = row_4['description_translated']

# Check if the text is valid before translation
if text_to_translate is None:
    print("Error: The text in row 4 is None")
    text_to_translate = "No description available"  # Provide a fallback value
elif isinstance(text_to_translate, str):
    # Ensure it's a string and clean it up using regex
    text_to_translate = re.sub(r'\s+', ' ', text_to_translate)  # Replace multiple spaces with a single space
    text_to_translate = re.sub(r'\n|\r', ' ', text_to_translate)  # Replace newlines with spaces
    text_to_translate = text_to_translate.strip()  # Trim leading and trailing spaces
    text_to_translate = re.sub(r'[^\x00-\x7F]+', ' ', text_to_translate)  # Remove non-ASCII characters (if needed)
else:
    # If it's not a string, log the type and assign a fallback value
    print(f"Error: Unexpected type in row 4. Found type: {type(text_to_translate)}. Using fallback text.")
    text_to_translate = "No description available"

# Print the restructured text before sending it to the translator
print(f"Text to translate (restructured): {text_to_translate}")

# Detect the language of the text
detected_lang = detect(text_to_translate)
print(f"Detected language: {detected_lang}")

# If the text is not in English, translate it
if detected_lang != 'en':
    try:
        print(f"Attempting to translate text: {text_to_translate}")
        
        # Translate the text using the translate package
        translated_text = translator.translate(text_to_translate)

        # Check the translation result
        print(f"Original text: {text_to_translate}")
        print(f"Translated text: {translated_text}")

        # Update the DataFrame with the translated text
        df_temp_all.loc[4, 'description_translated'] = translated_text
        print(f"Updated 5th row: {df_temp_all.loc[4, 'description_translated']}")
    except Exception as e:
        print(f"Error during translation: {e}")
else:
    print("The text is already in English, no translation needed.")


Full content of row 4: lens_id                                                   000-664-023-248-319
publication_number                                                EP2434144B1
title                       Rotor blade or rotor blade segment for a wind ...
description                 Die Erfindung betrifft ein Rotorblatt oder ein...
IsNonEnglish_desc                                                        True
IsNonEnglish_claims                                                      True
claims                      System mit einem Rotorblatt (10) oder einem Ro...
description_translated      Die Erfindung betrifft ein Rotorblatt oder ein...
IsNonEnglish_desc_2                                                      True
description_translated_2    Die Erfindung betrifft ein Rotorblatt oder ein...
Name: 4, dtype: object
Text to translate (restructured): Die Erfindung betrifft ein Rotorblatt oder ein Rotorblattsegment f r eine Windenergieanlage, das mit einem Seil an einer Rotornabe oder an e

In [31]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip uninstall googletrans

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install googletrans==3.1.0a0


In [34]:
df_temp_all

,lens_id,publication_number,title,description,IsNonEnglish_desc,IsNonEnglish_claims,claims,description_translated,IsNonEnglish_desc_2,description_translated_2
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,BLADE MOUNTING The present invention concerns ...,False,False,CLAIMS: 1. An arrangement for in situ mounting...,BLADE MOUNTING The present invention concerns ...,False,BLADE MOUNTING The present invention concerns ...
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,TECHNICAL FIELD The present invention relates ...,False,False,1 . A speed reducer used in a yaw drive appara...,TECHNICAL FIELD The present invention relates ...,False,TECHNICAL FIELD The present invention relates ...
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,False,False,A wind turbine (1) having a rotor (4) with at ...,The present invention relates to a wind turbin...,False,The present invention relates to a wind turbin...
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,False,False,Method of updating software and/or firmware of...,Field of invention The present invention relat...,False,Field of invention The present invention relat...
4,000-664-023-248-319,EP2434144B1,Rotor blade or rotor blade segment for a wind ...,Die Erfindung betrifft ein Rotorblatt oder ein...,True,True,System mit einem Rotorblatt (10) oder einem Ro...,Die Erfindung betrifft ein Rotorblatt oder ein...,True,Die Erfindung betrifft ein Rotorblatt oder ein...
...,...,...,...,...,...,...,...,...,...,...
61395,196-473-373-722-750,EP2708743A1,WINDMILL,Technical Field The present invention relates ...,False,False,A wind turbine comprising:\n a rotor head; \n ...,Technical Field The present invention relates ...,False,Technical Field The present invention relates ...
61396,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,FIELD OF THE INVENTION The present invention r...,False,False,A method for installing a tower (1) formed by ...,FIELD OF THE INVENTION The present invention r...,False,FIELD OF THE INVENTION The present invention r...
61397,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,BACKGROUND Embodiments of the present specific...,False,False,"1 . An armature for an electrical machine, com...",BACKGROUND Embodiments of the present specific...,False,BACKGROUND Embodiments of the present specific...
61398,198-470-700-069-007,EP3376025A1,WIND TURBINE VARIABLE-PITCH CONTROL METHOD AND...,The present application claims priority to Chi...,False,False,"A pitch control method for a wind turbine, com...",The present application claims priority to Chi...,False,The present application claims priority to Chi...


In [35]:
df_final = df_temp_all.drop(['IsNonEnglish_desc', 'description', 'description_translated'], axis=1)

In [39]:
df_final = df_final.rename(columns={'description_translated_2': 'description'})

In [40]:
df_final

,lens_id,publication_number,title,IsNonEnglish_claims,claims,IsNonEnglish_desc_2,description
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,False,CLAIMS: 1. An arrangement for in situ mounting...,False,BLADE MOUNTING The present invention concerns ...
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,False,1 . A speed reducer used in a yaw drive appara...,False,TECHNICAL FIELD The present invention relates ...
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,False,A wind turbine (1) having a rotor (4) with at ...,False,The present invention relates to a wind turbin...
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,False,Method of updating software and/or firmware of...,False,Field of invention The present invention relat...
4,000-664-023-248-319,EP2434144B1,Rotor blade or rotor blade segment for a wind ...,True,System mit einem Rotorblatt (10) oder einem Ro...,True,Die Erfindung betrifft ein Rotorblatt oder ein...
...,...,...,...,...,...,...,...
61395,196-473-373-722-750,EP2708743A1,WINDMILL,False,A wind turbine comprising:\n a rotor head; \n ...,False,Technical Field The present invention relates ...
61396,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,False,A method for installing a tower (1) formed by ...,False,FIELD OF THE INVENTION The present invention r...
61397,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,False,"1 . An armature for an electrical machine, com...",False,BACKGROUND Embodiments of the present specific...
61398,198-470-700-069-007,EP3376025A1,WIND TURBINE VARIABLE-PITCH CONTROL METHOD AND...,False,"A pitch control method for a wind turbine, com...",False,The present application claims priority to Chi...


In [41]:
df_final = df_final[df_final['IsNonEnglish_claims'] != True]
df_final = df_final[df_final['IsNonEnglish_desc_2'] != True]


In [43]:
# Check if there are any True values left in 'IsNonEnglish_claims'
if df_final['IsNonEnglish_desc_2'].any():
    print("There are still True values left in 'IsNonEnglish_claims'.")
else:
    print("There are no True values left in 'IsNonEnglish_claims'.")


There are no True values left in 'IsNonEnglish_claims'.


In [44]:
df_final = df_final.drop(['IsNonEnglish_desc_2', 'IsNonEnglish_claims'], axis=1)

In [47]:
df_final

,lens_id,publication_number,title,claims,description
0,000-041-469-766-603,WO2010103086A2,BLADE MOUNTING,CLAIMS: 1. An arrangement for in situ mounting...,BLADE MOUNTING The present invention concerns ...
1,000-316-779-119-014,US20060205554A1,Speed reducer for use in yaw drive apparatus f...,1 . A speed reducer used in a yaw drive appara...,TECHNICAL FIELD The present invention relates ...
2,000-374-150-323-590,EP2458201A1,Wind turbine with hydraulic blade pitch system,A wind turbine (1) having a rotor (4) with at ...,The present invention relates to a wind turbin...
3,000-602-602-272-491,EP3770422A1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Method of updating software and/or firmware of...,Field of invention The present invention relat...
4,000-729-551-090-695,US9410531B2,Wind turbine generator,1. A wind turbine generator comprising: a nace...,TECHNICAL FIELD The present invention relates ...
...,...,...,...,...,...
51421,196-298-591-904-637,US20210075232A1,System and Method for Detecting Battery Faults...,1 . A method for detecting a fault in a direct...,FIELD The present disclosure relates generally...
51422,196-473-373-722-750,EP2708743A1,WINDMILL,A wind turbine comprising:\n a rotor head; \n ...,Technical Field The present invention relates ...
51423,196-757-053-718-829,EP4411091A1,METHOD FOR INSTALLING A TOWER AND TOWER INSTAL...,A method for installing a tower (1) formed by ...,FIELD OF THE INVENTION The present invention r...
51424,198-109-502-126-536,US20200350805A1,MAGNETIC SHIELD FOR A SUPERCONDUCTING GENERATOR,"1 . An armature for an electrical machine, com...",BACKGROUND Embodiments of the present specific...


In [46]:
df_final = df_final.reset_index(drop=True)

In [48]:
df_final.to_excel('patent_database_final_F03D.xlsx', index=False)

In [38]:
df_temp_all.to_excel('output_file_2_claimsX3_descX2_backup.xlsx', index=False)

#### Clean training excel

In [49]:
df_train = pd.read_excel('final_data/final_similarity_data_translations.xlsx')

In [50]:
df_train

,lens_id_x,claim_x,title_x,description_x,citation_doc_number,citation_lens_id,citation_phase,citation_category,in_our_db,lens_id_y,title_y,description_y,claim_y,claim_x_trans,description_y_trans
0,000-374-150-323-590,A wind turbine (1) having a rotor (4) with at ...,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,US2007217912,120-349-166-665-84X,SEA,"Y, D",True,120-349-166-665-84X,CONTROL UNIT FOR ROTOR BLADE ADJUSTMENT,CROSS-REFERENCE TO A RELATED APPLICATION The i...,1 . An electrohydraulic control unit for rotor...,A wind turbine (1) having a rotor (4) with at ...,CROSS-REFERENCE TO A RELATED APPLICATION The i...
1,000-602-602-272-491,Method of updating software and/or firmware of...,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,US2011145811,095-788-761-259-678,SEA,Y,True,095-788-761-259-678,METHOD AND SYSTEM FOR SOFTWARE UPDATE OF WIND ...,BACKGROUND OF THE INVENTION The present disclo...,1 . A method for installing software data at a...,Method of updating software and/or firmware of...,BACKGROUND OF THE INVENTION The present disclo...
2,001-272-910-808-558,A computer-based method for providing a soft s...,A COMPUTER-BASED METHOD AND A PREDICTION DEVIC...,The control of wind turbines can be considered...,US2014100703,157-662-824-962-162,SEA,X,True,157-662-824-962-162,METHOD FOR THE COMPUTER-SUPPORTED GENERATION O...,Method for the computer-aided generation of a ...,1 . A method for computer-aided generation of ...,A computer-based method for providing a soft s...,Method for the computer-aided generation of a ...
3,002-215-058-157-360,"1. A modular drive system (I,I',I' 1 ) for a t...","A MODULAR DRIVE SYSTEM, A LOAD UNIT AND A TEST...","A modular drive system, a load unit and a test...",EP2708738,066-843-097-284-249,ISR,A,True,066-843-097-284-249,Wind turbine,The present invention relates to a wind turbin...,A wind turbine having a yaw system that compri...,"1. A modular drive system (I,I',I' 1 ) for a t...",The present invention relates to a wind turbin...
4,002-847-069-458-789,A system for the erection of a tower (T) compr...,A SYSTEM FOR THE ERECTION OF A WIND TURBINE TO...,Background of the invention Due to the increas...,WO2018185111,180-717-094-409-013,SEA,A,True,180-717-094-409-013,HOISTING SYSTEM FOR INSTALLING A WIND TURBINE,Hoisting System for Installing a Wind Turbine ...,C L A I M S,A system for the erection of a tower (T) compr...,Hoisting System for Installing a Wind Turbine ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28274,180-811-430-280-652,1 . A connecting joint for attaching a wind tu...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,EP2551512,017-859-941-253-624,ISR,A,True,017-859-941-253-624,Wind turbine blade connector assembly,FIELD OF THE INVENTION The present invention r...,A wind turbine blade connector assembly (100) ...,1 . A connecting joint for attaching a wind tu...,FIELD OF THE INVENTION The present invention r...
28275,180-811-430-280-652,1 . A connecting joint for attaching a wind tu...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,EP2138716,120-106-328-683-092,ISR,A,True,120-106-328-683-092,Blade insert,Object of the invention The invention describe...,Blade insert coupled in the solid lamination l...,1 . A connecting joint for attaching a wind tu...,Object of the invention The invention describe...
28276,197-847-474-005-427,1. Method for assembling an offshore support s...,METHOD FOR ASSEMBLING AN OFFSHORE SUPPORT STRU...,Method for assembling an offshore support stru...,WO2011147472,187-844-090-869-529,ISR,"D, A",True,187-844-090-869-529,"A SEGMENTED JACKET CONSTRUCTION, IN PARTICULAR...","Description A segmented jacket construction, i...","1. A segmented jacket construction (1, 6, 10, ...",1. Method for assembling an offshore support s...,"Description A se

In [51]:
df_train_f = df_train.drop(['citation_phase', 'citation_doc_number', 'citation_lens_id', 'in_our_db', 'lens_id_y', 'lens_id_x', 'claim_x', 'description_y'], axis=1)

In [52]:
df_train_f 

,title_x,description_x,citation_category,title_y,claim_y,claim_x_trans,description_y_trans
0,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,"Y, D",CONTROL UNIT FOR ROTOR BLADE ADJUSTMENT,1 . An electrohydraulic control unit for rotor...,A wind turbine (1) having a rotor (4) with at ...,CROSS-REFERENCE TO A RELATED APPLICATION The i...
1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,Y,METHOD AND SYSTEM FOR SOFTWARE UPDATE OF WIND ...,1 . A method for installing software data at a...,Method of updating software and/or firmware of...,BACKGROUND OF THE INVENTION The present disclo...
2,A COMPUTER-BASED METHOD AND A PREDICTION DEVIC...,The control of wind turbines can be considered...,X,METHOD FOR THE COMPUTER-SUPPORTED GENERATION O...,1 . A method for computer-aided generation of ...,A computer-based method for providing a soft s...,Method for the computer-aided generation of a ...
3,"A MODULAR DRIVE SYSTEM, A LOAD UNIT AND A TEST...","A modular drive system, a load unit and a test...",A,Wind turbine,A wind turbine having a yaw system that compri...,"1. A modular drive system (I,I',I' 1 ) for a t...",The present invention relates to a wind turbin...
4,A SYSTEM FOR THE ERECTION OF A WIND TURBINE TO...,Background of the invention Due to the increas...,A,HOISTING SYSTEM FOR INSTALLING A WIND TURBINE,C L A I M S,A system for the erection of a tower (T) compr...,Hoisting System for Installing a Wind Turbine ...
...,...,...,...,...,...,...,...
28274,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,A,Wind turbine blade connector assembly,A wind turbine blade connector assembly (100) ...,1 . A connecting joint for attaching a wind tu...,FIELD OF THE INVENTION The present invention r...
28275,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,A,Blade insert,Blade insert coupled in the solid lamination l...,1 . A connecting joint for attaching a wind tu...,Object of the invention The invention describe...
28276,METHOD FOR ASSEMBLING AN OFFSHORE SUPPORT STRU...,Method for assembling an offshore support stru...,"D, A","A SEGMENTED JACKET CONSTRUCTION, IN PARTICULAR...","1. A segmented jacket construction (1, 6, 10, ...",1. Method for assembling an offshore support s...,"Description A segmented jacket construction, i..."
28277,A PITCH CONTROLLED WIND TURBINE,A Pitch Controlled Wind Turbine FIELD OF THE I...,"X, A, I",A PITCH CONTROLLED WIND TURBINE WITH BLADE CON...,24,1. A pitch controlled wind turbine comprising ...,A PITCH CONTROLLED WIND TURBINE WITH BLADE CON...


In [56]:
from langdetect import detect

# Function to detect if a text is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # Return False if language detection fails

# Apply the function to filter out non-English rows
df_train_f = df_train_f[df_train_f['claim_x_trans'].apply(is_english)]

# Reset index if needed
df_train_f.reset_index(drop=True, inplace=True)

# Check the result
df_train_f.head()

,title_x,description_x,citation_category,title_y,claim_y,claim_x_trans,description_y_trans
0,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,"Y, D",CONTROL UNIT FOR ROTOR BLADE ADJUSTMENT,1 . An electrohydraulic control unit for rotor...,A wind turbine (1) having a rotor (4) with at ...,CROSS-REFERENCE TO A RELATED APPLICATION The i...
1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,Y,METHOD AND SYSTEM FOR SOFTWARE UPDATE OF WIND ...,1 . A method for installing software data at a...,Method of updating software and/or firmware of...,BACKGROUND OF THE INVENTION The present disclo...
2,A COMPUTER-BASED METHOD AND A PREDICTION DEVIC...,The control of wind turbines can be considered...,X,METHOD FOR THE COMPUTER-SUPPORTED GENERATION O...,1 . A method for computer-aided generation of ...,A computer-based method for providing a soft s...,Method for the computer-aided generation of a ...
3,"A MODULAR DRIVE SYSTEM, A LOAD UNIT AND A TEST...","A modular drive system, a load unit and a test...",A,Wind turbine,A wind turbine having a yaw system that compri...,"1. A modular drive system (I,I',I' 1 ) for a t...",The present invention relates to a wind turbin...
4,A SYSTEM FOR THE ERECTION OF A WIND TURBINE TO...,Background of the invention Due to the increas...,A,HOISTING SYSTEM FOR INSTALLING A WIND TURBINE,C L A I M S,A system for the erection of a tower (T) compr...,Hoisting System for Installing a Wind Turbine ...


In [62]:
df_train_f

,title_x,description_x,citation_category,title_y,claim_y,claim_x,description_y
0,Wind turbine with hydraulic blade pitch system,The present invention relates to a wind turbin...,"Y, D",CONTROL UNIT FOR ROTOR BLADE ADJUSTMENT,1 . An electrohydraulic control unit for rotor...,A wind turbine (1) having a rotor (4) with at ...,CROSS-REFERENCE TO A RELATED APPLICATION The i...
1,UPDATING SOFTWARE AND/OR FIRMWARE OF A PLURALI...,Field of invention The present invention relat...,Y,METHOD AND SYSTEM FOR SOFTWARE UPDATE OF WIND ...,1 . A method for installing software data at a...,Method of updating software and/or firmware of...,BACKGROUND OF THE INVENTION The present disclo...
2,A COMPUTER-BASED METHOD AND A PREDICTION DEVIC...,The control of wind turbines can be considered...,X,METHOD FOR THE COMPUTER-SUPPORTED GENERATION O...,1 . A method for computer-aided generation of ...,A computer-based method for providing a soft s...,Method for the computer-aided generation of a ...
3,"A MODULAR DRIVE SYSTEM, A LOAD UNIT AND A TEST...","A modular drive system, a load unit and a test...",A,Wind turbine,A wind turbine having a yaw system that compri...,"1. A modular drive system (I,I',I' 1 ) for a t...",The present invention relates to a wind turbin...
4,A SYSTEM FOR THE ERECTION OF A WIND TURBINE TO...,Background of the invention Due to the increas...,A,HOISTING SYSTEM FOR INSTALLING A WIND TURBINE,C L A I M S,A system for the erection of a tower (T) compr...,Hoisting System for Installing a Wind Turbine ...
...,...,...,...,...,...,...,...
26812,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,A,Wind turbine blade connector assembly,A wind turbine blade connector assembly (100) ...,1 . A connecting joint for attaching a wind tu...,FIELD OF THE INVENTION The present invention r...
26813,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,JOINT FOR CONNECTING A WIND TURBINE ROTOR BLAD...,A,Blade insert,Blade insert coupled in the solid lamination l...,1 . A connecting joint for attaching a wind tu...,Object of the invention The invention describe...
26814,METHOD FOR ASSEMBLING AN OFFSHORE SUPPORT STRU...,Method for assembling an offshore support stru...,"D, A","A SEGMENTED JACKET CONSTRUCTION, IN PARTICULAR...","1. A segmented jacket construction (1, 6, 10, ...",1. Method for assembling an offshore support s...,"Description A segmented jacket construction, i..."
26815,A PITCH CONTROLLED WIND TURBINE,A Pitch Controlled Wind Turbine FIELD OF THE I...,"X, A, I",A PITCH CONTROLLED WIND TURBINE WITH BLADE CON...,24,1. A pitch controlled wind turbine comprising ...,A PITCH CONTROLLED WIND TURBINE WITH BLADE CON...


In [61]:
df_train_f = df_train_f.rename(columns={'claim_x_trans': 'claim_x'})
df_train_f = df_train_f.rename(columns={'description_y_trans': 'description_y'})

In [63]:
df_train_f.to_excel('data_train_final.xlsx', index=False)